In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import ast
import torch
import random

from sklearn.metrics import accuracy_score

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install elasticsearch==7.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 219 kB 4.2 MB/s 


In [4]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.9 MB 1.4 MB/s 


In [5]:
%%bash
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.1-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.1-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.1-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.1/
shasum -a 512 -c elasticsearch-oss-7.9.1-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.1-linux-x86_64.tar.gz: OK


In [6]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.1/bin/elasticsearch

Starting job # 0 in a separate thread.


In [7]:
import os
import time
# Sleep for few seconds to let the instance start.
time.sleep(20)

In [8]:
%%bash

ps -ef | grep elasticsearch

root         402     400  0 05:01 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.1/bin/elasticsearch
daemon       403     402 99 05:01 ?        00:00:19 /content/elasticsearch-7.9.1/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-17071744341045369321 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

In [9]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "339d16be5d97",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "vRKJZxuDR82V0CJnHwTGiQ",
  "version" : {
    "number" : "7.9.1",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "083627f112ba94dffc1232e8b42b73492789ef91",
    "build_date" : "2020-09-01T21:22:21.964974Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [10]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk

import pandas as pd
import numpy as np
import re

In [11]:

import json
import time
from functools import reduce
import operator

In [12]:
es = Elasticsearch([{'host':'localhost', 'port':9200}])

In [13]:
es.ping()

True

In [14]:
corpus3 = open('drive/MyDrive/ARC/ARC_Corpus.txt', 'r', encoding='utf-8')

In [15]:
def create_index(es_client):
    """ Creates an Elasticsearch index."""
    is_created = False
    # Index settings
    settings = {
        "settings": {
            "number_of_shards": 2,
            "number_of_replicas": 1
        },
        "mappings": {
            "dynamic": "true",
            "_source": {
            "enabled": "true"
            },
            "properties": {
                "body": {
                    "type": "text"
                }
            }
        }
    }
    print('Creating `corpus3` index...')
    try:
        if es_client.indices.exists('corpus3'):
            es_client.indices.delete(index='corpus3', ignore=[404])
        es_client.indices.create(index='corpus3', body=settings)
        is_created = True
        print('index `corpus3` created successfully.')
    except Exception as ex:
        print(str(ex))
    finally:
        return is_created
    return is_created

In [16]:
create_index(es)

Creating `corpus3` index...
index `corpus3` created successfully.


True

In [17]:
from elasticsearch.helpers import bulk
def index_batch(docs):
    """ Indexes a batch of documents."""
    requests = []
    for i, doc in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = 'corpus3'
        request["body"] = doc['body']
        requests.append(request)
    bulk(es, requests)

In [18]:
def index_data(es_client, data, BATCH_SIZE=100000):
    """ Indexs all the rows in data (python questions)."""
    docs = []
    count = 0
    for line in data:
        js_object = {}
        js_object['body'] = line
        docs.append(js_object)
        count += 1

        if count % BATCH_SIZE == 0:
            index_batch(docs)
            docs = []
            print('Indexed {} documents.'.format(count))
    if docs:
        index_batch(docs)
        print('Indexed {} documents.'.format(count))

    es_client.indices.refresh(index='corpus3')
    print("Done indexing.")

In [19]:
index_data(es,corpus3.readlines(), BATCH_SIZE=100000)

Indexed 100000 documents.
Indexed 200000 documents.
Indexed 300000 documents.
Indexed 400000 documents.
Indexed 500000 documents.
Indexed 600000 documents.
Indexed 700000 documents.
Indexed 800000 documents.
Indexed 900000 documents.
Indexed 1000000 documents.
Indexed 1100000 documents.
Indexed 1200000 documents.
Indexed 1300000 documents.
Indexed 1400000 documents.
Indexed 1500000 documents.
Indexed 1600000 documents.
Indexed 1700000 documents.
Indexed 1800000 documents.
Indexed 1900000 documents.
Indexed 2000000 documents.
Indexed 2100000 documents.
Indexed 2200000 documents.
Indexed 2300000 documents.
Indexed 2400000 documents.
Indexed 2500000 documents.
Indexed 2600000 documents.
Indexed 2700000 documents.
Indexed 2800000 documents.
Indexed 2900000 documents.
Indexed 3000000 documents.
Indexed 3100000 documents.
Indexed 3200000 documents.
Indexed 3300000 documents.
Indexed 3400000 documents.
Indexed 3500000 documents.
Indexed 3600000 documents.
Indexed 3700000 documents.
Indexed 38

In [20]:
def extract_question(question):
    '''
    This function will take a value in question column then, will split the string 
    at (A) or (1). After splitting we will take only first part which is the question.
    '''
    if '(A)' in question:
        x = question.split('(A)')[0]
    elif '(1)' in question:
        x = question.split('(1)')[0]

    return x

In [21]:
def extract_answers(question):
    '''
    This function will take a value in question column then, will split the string 
    at (A) or (1). After splitting we will take only first part which is the question.
    '''
    if '(A)' in question:
        x = '(A) ' + question.split('(A)')[1]
    elif '(1)' in question:
        x = '(1) ' + question.split('(1)')[1]

    return x

In [22]:
def data_generator(a):
    '''
    This function will accept a data point and returns a list of options
    converts the only_options list of options.

    Output will be as follows:

    options = [option_1, option_2, option_3, option_4, option_5]                   

    Note: If there's no option E then it will be written as 'None of the above'.
    '''
    options = []

    if '(A)' in a:
        option_1 = a.split('(B)')[0].replace('(A)','').lstrip()

        b = ' '.join(i for i in a.split() if i not in option_1)
        option_2 = b.split('(C)')[0].replace('(B)','').replace('(A)','').lstrip()

        c = ' '.join(i for i in b.split() if i not in option_2)
        option_3 = c.split('(D)')[0].replace('(C)','').replace('(B)','').replace('(A)','').lstrip()

        if '(D)' not in c:
            option_4 = 'None of the above'
            option_5 = 'None of the above'
        else:
            if '(E)' not in c:
                option_4 = c.split('(D)')[1].lstrip()
                option_5 = 'None of the above'
            else:
                d = ' '.join(i for i in c.split() if i not in option_3)
                option_4 = d.split('(E)')[0].replace('(D)','').replace('(C)','').replace('(B)','').replace('(A)','').lstrip()
                option_5 = d.split('(E)')[1].lstrip()
    else:
        option_1 = a.split('(2)')[0].replace('(1)','').lstrip()

        b = ' '.join(i for i in a.split() if i not in option_1)
        option_2 = b.split('(3)')[0].replace('(2)','').replace('(1)','').lstrip()

        c = ' '.join(i for i in b.split() if i not in option_2)
        option_3 = c.split('(4)')[0].replace('(3)','').replace('(2)','').replace('(1)','').lstrip()

        if '(4)' in c:
            option_4 = c.split('(4)')[1].lstrip()
        else:
            option_4 = 'None of the above'
        option_5 = 'None of the above'

    options = [option_1, option_2, option_3, option_4, option_5]

    return options


In [23]:
def get_context(question, options):
    '''
    This function will return a context after joining a question
    and its options separately. 
    For example: question + option_1
                 question + option_2
                 question + option_3
                 question + option_4
   '''
    scores = []
    sentences = []
    choices = []
    
    #Dividing options into separate parts.
    if '(A)' in options:

        split_1 = options.split('(B)')
        option_1 = split_1[0].replace('(A)','')
        choices.append(option_1)

        split_2 = split_1[1].split('(C)')
        option_2 = split_2[0]
        choices.append(option_2)

        if '(D)' not in split_2[1]:
            option_3 = split_2[1]
            choices.append(option_3)

        elif '(D)' in split_2[1]:
            split_3 = split_2[1].split('(D)')
            option_3 = split_3[0]
            choices.append(option_3)

            if '(E)' not in split_3[1]:
                option_4 = split_3[1]
                choices.append(option_4)

            else:
                split_4 = split_3[1].split('(E)')
                option_4 = split_4[0]
                choices.append(option_4)

                option_5 = split_4[1]
                choices.append(option_5)
    else:
        split_1 = options.split('(2)')
        option_1 = split_1[0].replace('(1)','')
        choices.append(option_1)

        split_2 = split_1[1].split('(3)')
        option_2 = split_2[0]
        choices.append(option_2)

        if '(4)' not in split_2[1]:
            option_3 = split_2[1]
            choices.append(option_3)

        elif '(4)' in split_2[1]:
            split_3 = split_2[1].split('(4)')
            option_3 = split_3[0]
            choices.append(option_3)

            if '(5)' not in split_3[1]:
                option_4 = split_3[1]
                choices.append(option_4)

            else:
                split_4 = split_3[1].split('(5)')
                option_4 = split_4[0]
                choices.append(option_4)

                option_5 = split_4[1]
                choices.append(option_5)
    
    for i in [choices]:
  
        query=question+f'{i}'
        search_start = time.time()
        search = {"size":20,"query": {"match": {"body": query}}}
        #print(search)
        response = es.search(index='corpus3', body=json.dumps(search))
        search_time = time.time() - search_start
        #print("{} total hits.".format(response["hits"]["total"]["value"]))
        #print("search time: {:.2f} ms".format(search_time * 1000))

        #Searching in the 'corpus' index.
        a = es.search(index='corpus3', body=json.dumps(search))

        for j in a['hits']['hits']:
            sentence = j['_source']
            out=list(sentence.values())
            out = reduce(operator.concat,out)
            sentences.append(out)

   
    
    context= ' '.join(sentences) #Selecting top 20 sentences for the context.
    
    return context

In [24]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 
     |████████████████████████████████| 596 kB 79.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [25]:
from transformers import T5ForConditionalGeneration,AdamW

model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Unified_QA_2/')

In [26]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [27]:
def run_model(input_ids, **generator_args):
    global tokenizer
    res = model.generate(input_ids, **generator_args)
    return [tokenizer.decode(x) for x in res]

In [28]:
device = 'cpu'

In [29]:
def predict(array_of_input_ids, options_list):
    '''
    This function takes list of string. Each string is a pair of question and 
    answers separated using \n(all lower cased). 
    For example: 
    which of the following is an example of a physical change? \n (a) lighting a match (b) breaking a glass (c) burning of gasoline (d) rusting of iron

    label_map = {'A':0,
                'B':1,
                'C':2,
                'D':3,
                'E':4,
                '1':0,
                '2':1,
                '3':2,
                '4':3,
                '5':4}

    This function will return predicted option. All answers are converted to 0, 1, 2, 3 and 4. 

    '''

    # Converting input ids to torch tensor of Long type.
    input_ids = array_of_input_ids.type(torch.LongTensor).to(device)

    pred = run_model(input_ids.reshape((1,-1)))[0] #Reshaping because model takes input like (batch_size, sequence_length)

    #Pred is in the form like '<pad> answer </s>'. So we have to remove <pad> and </s>.
    pred = pred.replace('<pad>','')
    pred = pred.replace('</s>','')
    pred = pred.lstrip()

    label = None
    for j in range(len(options_list)):
        if pred in options_list[j]:
            label = j

    if label == None:
        label = random.randint(0,4)       

    return pred, label

In [30]:
MAX_LEN = 128

In [31]:
def final_fun_1(question):
    '''
    The ARC dataset contains question in the following format:
    
    Which factor will most likely cause a person to develop a fever?  (A) a leg muscle relaxing after exercise \
    (B) a bacterial population in the bloodstream (C) several viral particles on the skin \
    (D) carbohydrates being digested in the stomach
    
    As we can see here in raw form question contains options within it.
    
    Thus we will take question in same format.
    '''
    only_question = extract_question(question)
    only_answers = extract_answers(question)
    options_list = data_generator(only_answers)
    
    context = get_context(only_question,only_answers)
  
    input_string = only_question + '\\n ' + only_answers + '\\n ' + context + ' </s>'
    
    input_ids = tokenizer.encode(input_string, truncation=True, max_length=MAX_LEN, return_tensors='pt')
    
    pred = predict(input_ids, options_list)
    
    return pred

In [ ]:
pred_1 = final_fun_1('Rocks are classified as igneous, metamorphic, or sedimentary according to (1) their color (2) their shape (3) how they formed (4) the minerals they contain')

In [ ]:
pred_1

('how they formed', 2)

In [34]:
pred_2 = final_fun_1('Many states require vehicles to be examined and to meet safety and pollution standards. What impact might vehicle inspections have on the environment? (A) The environment will not be polluted. (B) The environment will become more polluted. (C) Fewer pollutants will be released by vehicles. (D) Fewer pollutants will be produced by older vehicles.')

In [35]:
pred_2

('Fewer pollutants will be released by vehicles.', 2)

In [36]:
pred_3 = final_fun_1('When a compression wave travels through a medium, in what direction is the medium displaced? (A) upward (B) downward (C) in the same direction (D) in the opposite direction')

In [37]:
pred_3

('the same direction', 2)

In [38]:
pred_4 = final_fun_1('Which list gives the correct order of substances from the lowest melting point to the highest? (A) oxygen, water, iron (B) water, iron, oxygen (C) oxygen, iron, water (D) iron, oxygen, water')

In [39]:
pred_4

('oxygen, water, iron', 0)

In [52]:
pred_5 = final_fun_1('The radioactive isotope content of a rock can be used to identify which property of the rock? (A) the total mass of the rock (B) the rate at which the rock formed (C) the types of fossils that the rock contains (D) the amount of time passed since the rock formed')

In [53]:
pred_5

('amount of time passed since', 3)

In [59]:
def final_fun_2(questions, labels):
    '''
    The ARC dataset contains question in the following format:
    
    Which factor will most likely cause a person to develop a fever?  (A) a leg muscle relaxing after exercise \
    (B) a bacterial population in the bloodstream (C) several viral particles on the skin \
    (D) carbohydrates being digested in the stomach
    
    As we can see here in raw form question contains options within it.
    
    Thus we will take question in same format.
    '''
    pred_labels = []
    
    for i in range(len(questions)):
        
        only_question = extract_question(questions[i])
        only_answers = extract_answers(questions[i])
        options_list = data_generator(only_answers)

        context = get_context(only_question,only_answers)
  
        input_string = only_question + '\\n ' + only_answers + '\\n ' + context + ' </s>'

        input_ids = tokenizer.encode(input_string, truncation=True, max_length=MAX_LEN, return_tensors='pt')

        pred, label = predict(input_ids, options_list)
        
        pred_labels.append(label)
    
    accuracy = accuracy_score(labels, pred_labels)
    
    return accuracy

In [74]:
label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4,
             '1':0,
             '2':1,
             '3':2,
             '4':3,
             '5':4}


In [55]:
Challenge_test=pd.read_csv('ARC-Challenge-Train.csv')

In [ ]:
questions =Challenge_test['question']

In [ ]:

labels = Challenge_test['AnswerKey'].map(label_map)

In [ ]:
accuracy_1 = final_fun_2(questions,labels)

In [ ]:
accuracy_1

0.5755138516532619

In [64]:
questions =list(Challenge_test['question'][10:14])

In [65]:
labels=[3,1,2,2]

In [66]:
accuracy_2 = final_fun_2(questions,labels)

In [67]:
accuracy_2

0.75

In [69]:
questions =list(Challenge_test['question'][109:114])

In [68]:
labels=[3,0,0,3,0]

In [70]:
accuracy_3 = final_fun_2(questions,labels)

In [71]:
accuracy_3

0.6

In [85]:
questions =list(Challenge_test['question'][250:300])

In [86]:
labels = Challenge_test['AnswerKey'][250:300].map(label_map)

In [87]:
accuracy_4 = final_fun_2(questions,labels)

In [88]:
accuracy_4

0.64

In [81]:
questions =list(Challenge_test['question'][600:700])

In [82]:
labels = Challenge_test['AnswerKey'][600:700].map(label_map)

In [83]:
accuracy_5 = final_fun_2(questions,labels)

In [84]:
accuracy_5

0.63